In [1]:
from tensorflow.keras.models import load_model

model = load_model('best_model', compile=False)  # Load the trained model


2023-06-10 20:05:37.481481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-06-10 20:05:40.416485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

image_folder = './images/test'  # Folder containing the images for prediction
# Iterate over each image in the folder
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image = load_img(image_folder)
    mask = model.predict(np.array([img_to_array(image)]))
    
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Original Image')

    # Display the predicted mask
    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap='gray')  # Assuming the mask is a grayscale image
    plt.title('Predicted Mask')

    plt.show()

IsADirectoryError: [Errno 21] Is a directory: './images/test'

In [3]:
image_folder = './images/test' 
path = os.path.join(image_folder, ????????)
image = load_img(path)
mask = model.predict(np.array([img_to_array(image)]))
plt.imshow(image)
plt.imshow(mask[0], alpha=0.3, cmap='jet')  # Display mask as an overlay
plt.show()


IsADirectoryError: [Errno 21] Is a directory: './images/test'